<img width="8%" alt="Data.gouv.fr.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Data.gouv.fr.png" style="border-radius: 15%">

# Data.gouv.fr - Visualisation de la carte de France des établissements éducatifs
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Data.gouv.fr+-+Visualisation+de+la+carte+de+France+des+établissements+éducatifs:+Error+short+description">Bug report</a>

**Tags:** #Data.gouv.fr

**Author:** [Paul Gilbert](https://www.linkedin.com/in/paul-gilbert-410783aa/)

**Last update:** 2024-02-29 (Created: 2024-02-29)

**Description:** This notebook demonstrates how to extract schools information ("identifiant", "latitude" and "longitude" from Data.gouv.fr API and then visualize it on a map with plotly.express.

**References:**
- [Data education.gouv.fr](https://data.education.gouv.fr/explore/dataset/fr-en-annuaire-education/table/?disjunctive.type_etablissement&disjunctive.code_postal&disjunctive.nom_commune&disjunctive.appartenance_education_prioritaire&disjunctive.libelle_academie&disjunctive.libelle_region&disjunctive.ministere_tutelle)
- [Naas Documentation](https://site.naas.ai/)

## Input

### Import libraries

In [ ]:
import pandas as pd
import requests
import json
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

### Setup variables
- `url`: Link to data source in JSON.
- `output_csv`: CSV output path.

In [ ]:
url = 'https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-annuaire-education/exports/json'
output_csv = "Education_France.csv"

## Model

### Get data from 'data.education.gouv.fr'

In [ ]:
def get_data(url):                                                                            
    res = requests.get(url)
    if res.status_code == 200:
        return pd.DataFrame(res.json())
    
df = get_data(url)
print('Etablissements:', len(df))
df.head(3)

### Visualize the schools on a map with plotly.express

In [ ]:
# Create dropdown widget
dropdown_type = widgets.Dropdown(
    options=df['type_etablissement'].unique().tolist(),
    value='Ecole',
    description="Type d'établissement"
)

dropdown_region = widgets.Dropdown(
    options=sorted(df.libelle_region.unique().tolist()),
    value='Ile-de-France',
    description="Région"
)

# Function to update plot
def update_plot(
    type_etablissement,
    region,
):
    filtered_df = df[(df['type_etablissement'] == type_etablissement) & (df['libelle_region'] == region)]
    fig = px.scatter_mapbox(
        filtered_df, 
        lat="latitude", 
        lon="longitude",
        color="nombre_d_eleves",
        color_continuous_scale=px.colors.cyclical.IceFire,
        zoom=6, 
        height=1200,
        width=1200
    )
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

# Use widgets.interact to create the interactive plot
widgets.interact(update_plot, type_etablissement=dropdown_type, region=dropdown_region)

## Output

### Save DataFrame to csv

In [ ]:
df.to_csv(output_csv, index=False)